In [29]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

In [30]:
class SyntheticRegressionData(d2l.DataModule):  #@save
    """Synthetic data for linear regression."""
    '''
    w：线性模型的权重向量。
    b：线性模型的偏置项。
    noise：生成数据时添加的噪声水平，默认值为0.01。
    num_train：训练数据集的数据点数量，默认值为1000。
    num_val：验证数据集的数据点数量，默认值为1000。
    batch_size：批处理大小，默认值为32。
    '''

    def __init__(self,
                 w,
                 b,
                 noise=0.01,
                 num_train=1000,
                 num_val=1000,
                 batch_size=32):
        super().__init__()
        self.save_hyperparameters()
        n = num_train + num_val  # 计算总数据点数
        # 生成一个形状为(n, len(w))的张量X，其中n是数据点的总数，len(w)是特征的数量。
        # 这里使用torch.randn生成标准正态分布（均值为0，方差为1）的随机数作为特征数据
        self.X = torch.randn(n, len(w))
        noise = torch.randn(n, 1) * noise
        self.y = torch.matmul(self.X, w.reshape((-1, 1))) + b + noise

    def get_dataloader(self, train):
        if train:
            indices = list(range(0, self.num_train))
            # The examples are read in random order
            random.shuffle(indices)
        else:
            indices = list(range(self.num_train,
                                 self.num_train + self.num_val))
        for i in range(0, len(indices), self.batch_size):
            batch_indices = torch.tensor(indices[i:i + self.batch_size])
            yield self.X[batch_indices], self.y[batch_indices]

    def get_tensorloader(self, tensors, train, indices=slice(0, None)):
        '''
        定义了一个名为get_tensorloader的方法，
        接收参数tensors（一个包含一个或多个张量的元组），
        train（布尔值，指示是否为训练模式，影响数据是否被打乱），
        以及indices（用于从张量中选择数据的索引，默为全选）。
        '''
        print('indices', indices)
        print('get_tensorloader 1   tensors:', tensors)
        # 下面的a 即是X，y，将X，y切片后，重新组成tuple
        tensors = tuple(a[indices] for a in tensors)
        print('get_tensorloader 2   tensors:', tensors)
        #使用索引后的张量创建了一个TensorDataset对象。TensorDataset是PyTorch中的一个数据集类，它能够包装特征张量和标签张量，使得两者能够一起被DataLoader加载
        dataset = torch.utils.data.TensorDataset(*tensors)
        #这行代码创建了一个DataLoader对象。这个对象从dataset中按照self.batch_size指定的批量大小加载数据。如果train为True，则通过shuffle=train参数打乱数据，以支持训练时的随机性。
        return torch.utils.data.DataLoader(dataset,
                                           self.batch_size,
                                           shuffle=train)

    def get_dataloader(self, train):
        i = slice(0, self.num_train) if train else slice(self.num_train, None)
        return self.get_tensorloader((self.X, self.y), train, i)

In [31]:
data = SyntheticRegressionData(w=torch.tensor([2, -3.4]), b=4.2)
print('features:', data.X[0],'\nlabel:', data.y[0])
print('features:', data.X)

features: tensor([-0.5385,  1.0862]) 
label: tensor([-0.5611])
features: tensor([[-0.5385,  1.0862],
        [ 1.8050,  0.2952],
        [ 0.5277, -0.7342],
        ...,
        [-0.1380,  0.1934],
        [ 1.4930,  1.2504],
        [-0.3382, -0.1487]])


In [32]:
X, y = next(iter(data.train_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)

indices slice(0, 1000, None)
get_tensorloader 1   tensors: (tensor([[-0.5385,  1.0862],
        [ 1.8050,  0.2952],
        [ 0.5277, -0.7342],
        ...,
        [-0.1380,  0.1934],
        [ 1.4930,  1.2504],
        [-0.3382, -0.1487]]), tensor([[-0.5611],
        [ 6.8217],
        [ 7.7492],
        ...,
        [ 3.2626],
        [ 2.9329],
        [ 4.0428]]))
get_tensorloader 2   tensors: (tensor([[-0.5385,  1.0862],
        [ 1.8050,  0.2952],
        [ 0.5277, -0.7342],
        ...,
        [-0.1380,  0.1934],
        [ 1.4930,  1.2504],
        [-0.3382, -0.1487]]), tensor([[-0.5611],
        [ 6.8217],
        [ 7.7492],
        ...,
        [ 3.2626],
        [ 2.9329],
        [ 4.0428]]))
X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])
